In [24]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import PIL.Image as Image

Get data

In [25]:
IMAGE_SIZE = (28, 28)
EPOCH = 5
# When weights become NaN, try reducing the learning rate
LEARNING_RATE = 0.00001

# Import data
data = pd.read_csv(r"../digit-recognizer/train.csv")
data_test = pd.read_csv(r"../digit-recognizer/test.csv")    

# Get labels and image array from data
labels : np.ndarray = data.values[:, 0]
images : np.ndarray = data.values[:, 1:].astype('uint8')

images_test : np.ndarray = data_test.values.astype('uint8')

KeyboardInterrupt: 

Process data

# Test sofmax

In [ ]:
class Model(nn.Module):
    def __init__(self, input, hidden, output):
        super().__init__()
        self.layer1 = nn.Linear(input, output)
        # self.layer2 = nn.Linear(hidden, output)

    def forward(self, x):
        x = self.layer1(x)
        # x = self.layer2(x)
        # torch.sigmoid(x)

        return x

In [ ]:
model = Model(IMAGE_SIZE[0] * IMAGE_SIZE[1], 1024, 10)
# Cross Entropy Loss already includes softmax
criterion = nn.CrossEntropyLoss()   
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
for i in range(EPOCH):
    wrong = 0
    for j in range(len(images)):
        optimizer.zero_grad()
        res = model(torch.tensor(images[j] / 255, dtype=torch.float64))
        if (int(torch.argmax(res)) != labels[j]):
            wrong += 1

        # Create label for calculating loss, for sigmoid
        # label_tensor = np.zeros(10)
        # label_tensor[labels[j]] = 1
        # label_tensor = torch.tensor(label_tensor, dtype=torch.float32)
        # print(label_tensor)

        # CrossEntropyLoss, target tensor only contains one label output
        label_tensor = torch.tensor([labels[j]], dtype=torch.long)
        res = torch.reshape(res, (1, 10))
        loss = criterion(res, label_tensor)
        loss.backward()
        optimizer.step()
        
        # print(model.layer1.weight)
        if ((j+1) % 2000 == 0):
            print(f"Data {j+1}: Wrong = {wrong}, Accuracy: {100-wrong/j*100}%")
        
    print(f"Epoch: {i} --> Wrong: {wrong}, Accuracy: {100-wrong / labels.size * 100}%\n")    

# Test model
wrong = 0
for i in range(100):
    res = torch.argmax(model(torch.tensor(images_test[i], dtype=torch.float32)))
    img = Image.fromarray(images_test[i].reshape(IMAGE_SIZE))
    print(f"Image {i+1}: {res}")
    img.save(f"./test_softmax/{str(i+1)}_ans={res}.jpg")


Data 2000: Wrong = 1675, Accuracy: 16.208104052026016%
Data 4000: Wrong = 3208, Accuracy: 19.779944986246562%
Data 6000: Wrong = 4704, Accuracy: 21.58693115519253%
Data 8000: Wrong = 6132, Accuracy: 23.340417552194026%
Data 10000: Wrong = 7549, Accuracy: 24.502450245024505%
Data 12000: Wrong = 8858, Accuracy: 26.177181431785982%
Data 14000: Wrong = 10094, Accuracy: 27.894849632116575%
Data 16000: Wrong = 11286, Accuracy: 29.458091130695678%
Data 18000: Wrong = 12452, Accuracy: 30.818378798822152%
Data 20000: Wrong = 13541, Accuracy: 32.29161458072903%
Data 22000: Wrong = 14634, Accuracy: 33.47879449065867%
Data 24000: Wrong = 15660, Accuracy: 34.74728113671402%
Data 26000: Wrong = 16657, Accuracy: 35.93215123658602%
Data 28000: Wrong = 17659, Accuracy: 36.9298903532269%
Data 30000: Wrong = 18597, Accuracy: 38.00793359778659%
Data 32000: Wrong = 19482, Accuracy: 39.116847401481294%
Data 34000: Wrong = 20374, Accuracy: 40.074708079649405%
Data 36000: Wrong = 21216, Accuracy: 41.065029584

KeyboardInterrupt: 

# Test sigmoid

In [ ]:
class ModelSigmoid(nn.Module):
    def __init__(self, input, hidden, output):
        super().__init__()
        self.layer1 = nn.Linear(input, output)
        # self.layer2 = nn.Linear(hidden, output)

    def forward(self, x):
        x = self.layer1(x)
        # x = self.layer2(x)
        torch.sigmoid(x)

        return x

In [28]:
model = ModelSigmoid(IMAGE_SIZE[0] * IMAGE_SIZE[1], 1024, 10)
criterion = nn.MSELoss()   
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
for i in range(EPOCH):
    wrong = 0
    for j in range(len(images)):
        optimizer.zero_grad()
        res = model(torch.tensor(images[j] / 255.0, dtype=torch.float32))
        if (int(torch.argmax(res)) != labels[j]):
            wrong += 1

        # Create label for calculating loss, for sigmoid
        label_tensor = np.zeros(10)
        label_tensor[labels[j]] = 1
        label_tensor = torch.tensor(label_tensor, dtype=torch.float32)
        
        loss = criterion(res, label_tensor)
        loss.backward()
        optimizer.step()
        
        # print(model.layer1.weight)
        if ((j+1) % 2000 == 0):
            print(f"Data {j+1}: Wrong = {wrong}, Accuracy: {100-wrong/j*100}%")
        
    print(f"Epoch: {i} --> Wrong: {wrong}, Accuracy: {100-wrong / labels.size * 100}%\n")    

# Test model
wrong = 0
for i in range(100):
    res = torch.argmax(model(torch.tensor(images_test[i], dtype=torch.float32)))
    img = Image.fromarray(images_test[i].reshape(IMAGE_SIZE))
    print(f"Image {i+1}: {res}")
    img.save(f"./test_sigmoid/{str(i+1)}_ans={res}.jpg")

Data 2000: Wrong = 1877, Accuracy: 6.103051525762879%
Data 4000: Wrong = 3731, Accuracy: 6.70167541885472%
Data 6000: Wrong = 5538, Accuracy: 7.684614102350395%
Data 8000: Wrong = 7337, Accuracy: 8.276034504313031%
Data 10000: Wrong = 9141, Accuracy: 8.580858085808586%
Data 12000: Wrong = 10951, Accuracy: 8.734061171764324%
Data 14000: Wrong = 12719, Accuracy: 9.143510250732191%
Data 16000: Wrong = 14486, Accuracy: 9.456841052565792%
Data 18000: Wrong = 16219, Accuracy: 9.8894383021279%
Data 20000: Wrong = 17951, Accuracy: 10.240512025601276%
Data 22000: Wrong = 19644, Accuracy: 10.705032046911228%
Data 24000: Wrong = 21362, Accuracy: 10.98795783157631%
Data 26000: Wrong = 23066, Accuracy: 11.281203123197045%
Data 28000: Wrong = 24774, Accuracy: 11.518268509589618%
Data 30000: Wrong = 26401, Accuracy: 11.993733124437483%
Data 32000: Wrong = 28060, Accuracy: 12.309759679989995%
Data 34000: Wrong = 29705, Accuracy: 12.629783228918498%
Data 36000: Wrong = 31310, Accuracy: 13.0253618156059

KeyboardInterrupt: 